In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal

sns.set_context("notebook")
sns.set_style("ticks")
sns.set_palette("colorblind")

In [ ]:
def decaying_sinusoid(t, amp, decay, frequency, phase):
    return amp * np.exp(-decay * t) * np.sin(2 * np.pi * frequency * t + phase)

In [ ]:
t_obs = 100
fs = 20_000
nperfft = fs // 2
time = np.linspace(0, t_obs, int(fs * t_obs))

f_1 = 2800.0
f_2 = 2801.0

In [ ]:
sig_1 = decaying_sinusoid(time, 50, 0.05, f_1, np.pi / 4)
sig_2 = decaying_sinusoid(time, 51, 0.1001, f_2, np.pi / 4)

In [ ]:
sig = sig_1 + sig_2
noisy = sig + 1 * np.random.randn(sig.size)

In [ ]:
plt.plot(time, noisy)
plt.plot(time, sig)
plt.xlabel("Time [s]")
plt.ylabel("Amplitude")
plt.show()

In [ ]:
psd = np.abs(np.fft.rfft(sig)) ** 2
freqs = np.fft.rfftfreq(len(sig), d=1 / fs)
idx = np.argsort(freqs)

In [ ]:
plt.plot(freqs[idx], psd[idx])
plt.xlabel("Frequency [Hz]")
plt.ylabel("PSD")
plt.show()

In [ ]:
f, t, Zxx = signal.stft(sig, fs=fs, nperseg=nperfft, noverlap=0, scaling="psd")
f_n, t_n, Zxx_n = signal.stft(noisy, fs=fs, nperseg=nperfft, noverlap=0, scaling="psd")

In [ ]:
print("Frequency bins, time steps: ", Zxx.shape)
print(f"Frequency resolution:  {fs / Zxx.shape[0]} Hz")

plt.pcolormesh(t, f, np.abs(Zxx), shading='gouraud')
plt.title('STFT Magnitude')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.colorbar(label="PSD")
plt.xlim([0, 20])
plt.ylim([1900, 2100])
plt.show()

In [ ]:
plt.plot(time[::nperfft], np.max(Zxx_n, axis=0)[:-1])
plt.xlabel("Time [s]")
plt.ylabel("Max PSD")
plt.xlim([0, 50])
plt.show()

In [ ]:
plt.plot(time[::nperfft], np.max(Zxx_n, axis=0)[:-1] - np.max(Zxx, axis=0)[:-1])